In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing
from filterpy.kalman import KalmanFilter

# Erstellen von Beispieldaten
np.random.seed(42)
dates = pd.date_range(start="2019-01-01", periods=1000, freq='D')
data = 20 + 5 * np.sin(np.linspace(0, 20, 1000)) + np.random.normal(0, 2, 1000)

# Daten als pandas DataFrame
df = pd.DataFrame({'date': dates, 'value': data})


# Daten als pandas DataFrame

# Plotten der Beispieldaten
plt.figure(figsize=(12, 6))
plt.plot(df.date, df['value'], label='Original Data')
plt.legend()
plt.show()



In [ ]:

df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.strip()
df

In [ ]:
series.components

In [ ]:
series["value"]

In [ ]:
# Holt-Winters Exponential Smoothing mit darts
series = TimeSeries.from_dataframe(df, 'date', 'value')
model = ExponentialSmoothing(seasonal_periods=365)
model

In [ ]:



model.fit(series)
hw_forecast = model.predict(len(series))

# Plotten der HWT-ES Vorhersage
plt.figure(figsize=(12, 6))
# series.plot(label='Original Data')
hw_forecast.plot(label='Holt-Winters Forecast', lw=3)
plt.legend()
plt.show()



In [ ]:
# Kalman-Filter
# Initialisierung des Kalman-Filters
kf = KalmanFilter(dim_x=2, dim_z=1)
kf.x = np.array([0., 1.])  # Initialer Zustand (Level und Trend)
kf.F = np.array([[1., 1.], [0., 1.]])  # Übergangsmatrix
kf.H = np.array([[1., 0.]])  # Messmatrix
kf.P *= 1000.  # Kovarianzmatrix des Schätzfehlers
kf.R = 5  # Messrauschen
kf.Q = np.array([[0.1, 0.1], [0.1, 0.1]])  # Prozessrauschen

kalman_forecast = []
for value in df['value']:
    kf.predict()
    kf.update([value])
    kalman_forecast.append(kf.x[0])

# Plotten der Kalman-Filter Vorhersage
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value'], label='Original Data')
plt.plot(df.index, kalman_forecast, label='Kalman Filter Forecast')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing
from filterpy.kalman import KalmanFilter

# Erstellen von Beispieldaten

# Plotten der Beispieldaten
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['value'], label='Original Data')
plt.legend()
plt.show()

# Holt-Winters Exponential Smoothing mit darts
series = TimeSeries.from_dataframe(df, time_col='date', value_cols='value')
model = ExponentialSmoothing( seasonal_periods=365)
model.fit(series)
hw_forecast = model.predict(len(series))

# Plotten der HWT-ES Vorhersage
plt.figure(figsize=(12, 6))
series.plot(label='Original Data')
hw_forecast.plot(label='Holt-Winters Forecast')
plt.legend()
plt.show()

# Holen der letzten Werte für Initialisierung des Kalman-Filters
last_level = series.values()[-1][0]
last_trend = (series.values()[-1][0] - series.values()[-2][0])  # Einfache Schätzung des Trends

# Initialisierung des Kalman-Filters basierend auf HWT-ES
kf = KalmanFilter(dim_x=2, dim_z=1)  # Hier verwenden wir Level und Trend als Zustandsvariablen
kf.x = np.array([last_level, last_trend])  # Initialer Zustand basierend auf HWT-ES
kf.F = np.array([[1, 1], [0, 1]])  # Übergangsmatrix angepasst
kf.H = np.array([[1, 0]])  # Messmatrix, die nur Level misst
kf.P *= 1000.  # Kovarianzmatrix des Schätzfehlers
kf.R = 5  # Messrauschen
kf.Q = np.eye(2) * 0.1  # Prozessrauschen

kalman_forecast = []
for value in df['value']:
    kf.predict()
    kf.update([value])
    kalman_forecast.append(kf.x[0])  # Level

# Plotten der Kalman-Filter Vorhersage
plt.figure(figsize=(12, 6))
# plt.plot(df['date'], df['value'], label='Original Data')
plt.plot(df['date'], kalman_forecast, label='Kalman Filter Forecast')
plt.legend()
plt.show()


In [ ]:
len(series.values())

In [ ]:
last_level

In [ ]:
import numpy as np
from pykalman import KalmanFilter

# Generate synthetic data (replace with your actual data)
n_timesteps = 1000
t = np.arange(n_timesteps)
data = 10 + 0.5 * t + 10 * np.sin(2 * np.pi * t / 50) + np.random.normal(0, 1, n_timesteps)

# Initial estimates for level, trend, and seasonality
initial_state_mean = [data[0], 0, 0]

# State transition matrix
transition_matrix = [[1, 1, 0],
                     [0, 1, 0],
                     [0, 0, 1]]

# Observation matrix
observation_matrix = [1, 0, 1]

# Kalman Filter
kf = KalmanFilter(transition_matrices=transition_matrix,
                  observation_matrices=observation_matrix,
                  initial_state_mean=initial_state_mean)

# Fit and smooth
state_means, state_covariances = kf.smooth(data)

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
plt.plot(data, label='Observed')
plt.plot(state_means[:, 0], label='Level (l_t)')
plt.plot(state_means[:, 2], label='Seasonality (s_t)')
plt.legend()
plt.show()
